# Setup

In [127]:
import pandas as pd
import numpy as np
import requests as re
import csv

In [128]:
# create empty csv file
table_csv = open('table_csv.csv','w')
csv_headers = csv.writer(table_csv)
csv_headers.writerow(['Postcode', 'Borough','Neighbourhood'])

32

# Scraping

In [129]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


### Remove "Not assigned" boroughs

In [130]:
df = df[df['Borough'] != 'Not assigned'].reset_index(drop=True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Not assigned
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [131]:
# Count rows
n = len(df)
n

210

### Combine Neighbourhood names if in same Borough

In [132]:
# Create new table for combined Neighbourhood names
df_comb = df.groupby(['Postcode', 'Borough'], as_index = False).agg(lambda x: ",".join(x))
print(len(df_comb))
df_comb.head(10)

103


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


### Give "Not assigned" neighbourhoods name of Borough

In [133]:
# Find any Neighbourhoods with 'Not assigned' value
NANb = df_comb[df_comb['Neighbourhood'] == 'Not assigned']
NANb

,Postcode,Borough,Neighbourhood
93,M9A,Queen's Park,Not assigned


In [134]:
# Replace 'Not assigned' values
for index, row in df_comb.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']

# Check if Queen's Park has changed
df_comb.iloc[93,:]

Postcode                  M9A
Borough          Queen's Park
Neighbourhood    Queen's Park
Name: 93, dtype: object

### Dataframe shape

In [135]:
df_comb.shape

(103, 3)